In [1]:
import numpy as np
import pandas as pd
# import nltk
# import re

# import gspread
# from gspread.exceptions import WorksheetNotFound
# from gspread_dataframe import get_as_dataframe, set_with_dataframe


from Levenshtein import ratio
from fuzzy import Soundex
# import scipy.sparse as sps
# import scipy.cluster.hierarchy as sch

from collections import Counter

# import spacy
# from spacy.lemmatizer import Lemmatizer
# from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Whisker, Plot
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, \
BoxZoomTool, WheelZoomTool, ResetTool, PanTool, ColumnDataSource, LabelSet, FactorRange
from bokeh.models.graphs import from_networkx

from bokeh.layouts import gridplot



# from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, \
# BoxZoomTool, WheelZoomTool, ResetTool, PanTool, ColumnDataSource, LabelSet

# from bokeh.plotting import figure
# from bokeh.transform import factor_cmap
from bokeh.palettes import Category20_20, Accent

# from bokeh.layouts import row

# from sklearn.feature_extraction.text import CountVectorizer

import networkx as nx

# from bokeh.models.graphs import from_networkx

import random
import itertools
import math

import pprint
pp = pprint.PrettyPrinter(indent=4)




# from iteration_utilities import flatten

# import matplotlib as mpl
# import matplotlib.pyplot as plt

In [29]:
depositions_df = pd.read_json('../converted_json/all_depositions.json')


In [23]:
pd.__version__


'0.23.4'

In [30]:
depositions_df

,country,creation_date,deponent_county,deponent_town,deponent_town_geonames_fuzziness,deponent_town_geonames_id,deponent_town_lat,deponent_town_lng,deposition_type,filename,nature_of_deposition,participants_number,people_list,signed_by,title
0,Ireland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dep_809000r000_tei.xml,[Titlepage],0,[],NaN,NaN
1,Ireland,1642-3-3,Unknown,,NaN,NaN,NaN,NaN,[Waring Copy],dep_809001r001_tei.xml,[words],14,"[{'role': 'Deponent', 'forename': 'Henry', 'su...","[{'role': 'Commissioner', 'name': 'William Ald...",NaN
2,Ireland,1642-8-22,Armagh,Tinon,0.6,NaN,NaN,NaN,[Dublin Original],dep_809005r002_tei.xml,"[apostacy, arson, assault, Captivity, Killing,...",51,"[{'role': 'Deponent', 'forename': 'Robert', 's...","[{'role': 'Commissioner', 'name': 'William Ald...",NaN
3,Ireland,1641-10-22,Unknown,,NaN,NaN,NaN,NaN,NaN,dep_809013r003_tei.xml,[words],4,"[{'role': 'Deponent', 'forename': 'Owen', 'sur...","[{'role': 'Commissioner', 'name': 'Rob: Meredi...",NaN
4,Ireland,1641-10-23,Monaghan,Connagh,0.6,2965167.0,52.66306,-8.68944,NaN,dep_809015r004_tei.xml,NaN,3,"[{'role': 'Deponent', 'forename': 'Hughe', 'su...","[{'role': 'Commissioner', 'name': 'Rob Meredit...",NaN
5,Ireland,1641-10-23,Cavan,Tirri Laghada,0.6,NaN,NaN,NaN,NaN,dep_809017r005a_tei.xml,[Investigative],7,"[{'role': 'Deponent', 'forename': 'Hugh', 'sur...","[{'role': 'Commissioner', 'name': 'Gerrard Low...",NaN
6,Ireland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dep_809018r006_tei.xml,"[Captivity, Multiple Killing, words]",16,"[{'role': 'Rebel', 'forename': 'Lord', 'surnam...",NaN,NaN
7,Ireland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dep_809022r007_tei.xml,"[arson, Captivity, Robbery, words]",15,"[{'role': 'Rebel', 'forename': 'Colloe', 'surn...",NaN,NaN
8,Ireland,1642-3-26,Fermanagh,Eniskellen,0.6,2964402.0,51.74167,-8.93806,NaN,dep_809024r008_tei.xml,[words],21,"[{'role': 'Deponent', 'forename': 'Lord', 'sur...","[{'role': 'Commissioner', 'name': 'Robert Mere...",NaN
9,Ireland,1641-10-23,Fermanagh,Knockninny,0.6,2963124.0,52.87312,-6.49672,NaN,dep_809026r009_tei.xml,[Investigative],8,"[{'role': 'Deponent', 'forename': 'Shane', 'su...","[{'role': 'Commissioner', 'name': 'Gerrard Low...",NaN


In [36]:
# depositions_df.groupby('deponent_county').count()
counts = depositions_df['deponent_county'].value_counts()


In [41]:
counts.min()

20

In [35]:
len(depositions_df['deponent_county']) - depositions_df['deponent_county'].count()

1131

In [22]:
depositions_df['creation_date_period'] = pd.PeriodIndex(depositions_df['creation_date'], freq='d')

In [6]:
depositions_df.dtypes

creation_date           object
creation_place          object
deposition_type         object
filename                object
nature_of_deposition    object
participants_number      int64
people_list             object
signed_by               object
title                   object
creation_date_period    object
dtype: object

In [7]:
type(depositions_df['deponent_town'][2])

KeyError: 'deponent_town'

In [8]:
depositions_df['creation_year'] = depositions_df['creation_date'].str.slice(start=0, stop=4)

In [9]:
depositions_df['creation_year'] = depositions_df['creation_year'].fillna('Unknown')

In [10]:
group_a = depositions_df.groupby('creation_year')

In [19]:
depositions_df.hist(column='creation_year')

ImportError: matplotlib is required for plotting.

In [11]:
group_a.describe()

participants_number                                           \
                            count       mean        std   min    25%   50%   
creation_year                                                                
1641                        145.0   5.496552   4.725998   0.0   3.00   4.0   
1642                       3124.0   8.501601  13.694543   0.0   2.00   5.0   
1643                        742.0   9.548518  17.634226   0.0   1.00   5.0   
1644                        231.0   8.614719  13.316010   0.0   0.00   4.0   
1645                         98.0  14.551020  52.305826   0.0   1.00   5.0   
1646                         50.0   8.520000  17.888818   0.0   1.00   3.0   
1647                         15.0   6.600000   6.957011   1.0   3.50   5.0   
1650                          2.0   6.000000   5.656854   2.0   4.00   6.0   
1652                        295.0   8.962712   7.276042   0.0   4.00   7.0   
1653                        972.0   7.471193   6.629071   0.0   3.00   6.0   
1654                        991.0   4.063572   3.817159   0.0   2.00   3.0   
1655                          4.0   5.500000   3.696846   2.0   2.75   5.0   
1656                          1.0  33.000000        NaN  33.0  33.00  33.0   
1660                          2.0   3.000000   0.000000   3.0   3.00   3.0   
1662                          4.0   8.000000   6.271629   3.0   3.00   6.5   
1667                          1.0   3.000000        NaN   3.0   3.00   3.0   
Unknown                     334.0   7.802395  16.810457   0.0   1.00   5.0   

                             
                 75%    max  
creation_year                
1641            7.00   39.0  
1642           10.00  210.0  
1643           11.00  253.0  
1644           10.00  101.0  
1645           12.00  488.0  
1646            7.00  100.0  
1647            7.00   29.0  
1650            8.00   10.0  
1652           11.00   55.0  
1653            9.00   66.0  
1654            5.00   35.0  
1655            7.75   10.0  
1656           33.00   33.0  
1660            3.00    3.0  
1662           11.50   16.0  
1667            3.00    3.0  
Unknown         9.00  261.0

# Number of depositions per year

In [12]:
output_notebook()

p = figure(plot_width=800, plot_height=600, x_range=group_a, title="Depositions by Year",
           toolbar_location=None, tools="")

p.vbar(x='creation_year', top='participants_number_count', width=0.8, source=group_a)

p.y_range.start = 0
p.xgrid.grid_line_color = None

show(p)

Loading BokehJS ...

# Number of participants per deposition

In [24]:
depositions_df['participants_number'].describe()

count    7011.000000
mean        7.848952
std        13.790283
min         0.000000
25%         2.000000
50%         5.000000
75%         9.000000
max       488.000000
Name: participants_number, dtype: float64

In [25]:
depositions_df[depositions_df['participants_number'] == 0]

,creation_date,creation_place,deposition_type,filename,nature_of_deposition,participants_number,people_list,signed_by,title,creation_date_period
0,NaN,{'country': 'Ireland'},NaN,dep_809000r000_tei.xml,[Titlepage],0,[],NaN,NaN,NaT
88,1641-11-12,{'country': 'Ireland'},[Other Copy],dep_809166r088_tei.xml,"[words, Investigative]",0,[],NaN,NaN,1641-11-12
89,1641-11-12,{'country': 'Ireland'},[Other Copy],dep_809168r089_tei.xml,"[words, Investigative]",0,[],NaN,NaN,1641-11-12
90,1641-11-12,{'country': 'Ireland'},[Other Copy],dep_809170r090_tei.xml,"[words, Investigative]",0,[],NaN,NaN,1641-11-12
92,1641-11-12,{'country': 'Ireland'},[Other Copy],dep_809174r092_tei.xml,"[words, Investigative]",0,[],NaN,NaN,1641-11-12
232,NaN,{'country': 'Ireland'},NaN,dep_810000r000a_tei.xml,[Title page],0,[],NaN,NaN,NaT
233,NaN,{'country': 'Ireland'},NaN,dep_810001r001_tei.xml,[Index],0,[],NaN,NaN,NaT
234,1642-2-5,{'country': 'Ireland'},[Waring Copy],dep_810003r002_tei.xml,NaN,0,[],NaN,NaN,1642-02-05
235,1642-2-26,{'country': 'Ireland'},[Waring Copy],dep_810003v003_tei.xml,NaN,0,[],NaN,NaN,1642-02-26
236,1642-2-28,{'country': 'Ireland'},[Waring Copy],dep_810004r004_tei.xml,NaN,0,[],NaN,NaN,1642-02-28


In [26]:
part_hist, edges = np.histogram(depositions_df['participants_number'], bins = list(range(0,50,1)))
participants = pd.DataFrame({'part_hist' : part_hist, 'left': edges[:-1], 'right': edges[1:]})   

In [27]:
participants

,part_hist,left,right
0,998,0,1
1,347,1,2
2,918,2,3
3,637,3,4
4,601,4,5
5,555,5,6
6,431,6,7
7,342,7,8
8,293,8,9
9,240,9,10


In [28]:
output_notebook()

p = figure(plot_width=800, 
           plot_height=600, 
           title="Histogram of Participants (up to 100) in 1641 Depositions",
           x_axis_label = "Number of participants",
           y_axis_label = "Number of depositions")

p.quad(bottom=0,
       top=participants['part_hist'],
       left=participants['left'],
       right=participants['right'],
       fill_color='red',
       line_color='black')

show(p)

Loading BokehJS ...

# People's names

In [ ]:
name_counter = Counter()
people_list = depositions_df['people_list'].tolist()
names_list = []
person2indx = {}
#All participants have only one role (checked)
indx2role = {}
for i, dep_part in enumerate(people_list):
    names = []
    for person in dep_part:
        person_str = ''
        if 'forename' in person:
            person_str += person['forename']
        else:
            person_str += 'None'
        person_str += ' '
        
        if 'surname' in person:
            person_str += person['surname']
        else:
            person_str += 'None'
    
        if person_str not in person2indx:
            idx = len(person2indx)
            person2indx[person_str] = idx
            indx2role[idx] = {i : person['role']}
            if person_str == 'Dauid Roch':
                print('1',person_str, person['role'], i, idx)
        else:
            idx = person2indx[person_str]
            indx2role[idx][i] = person['role']
            if person_str == 'Dauid Roch':
                print('2',person_str, person['role'], i)
        name_counter[person_str] += 1
        names.append(person_str)
    names_list.append(names)

indx2person = {v:k for k,v in person2indx.items()}

In [ ]:
for i,name in enumerate(name_counter.most_common(1000)):
    if name[0].split(' ')[0] == '*':
        print(i, name)

In [ ]:
sorted(list(filter(lambda x: x.split(' ')[0] == '*', name_counter.keys())))

In [ ]:
sorted(list(filter(lambda x: x.split(' ')[1] == 'Roch', name_counter.keys())))

# People network

In [ ]:
soundex = Soundex(4)

In [ ]:
G = nx.Graph()
for i, dep_names in enumerate(names_list):
    for person_a, person_b in itertools.combinations(dep_names, 2):
        G.add_edge(person2indx[person_a], person2indx[person_b], deposition=i)
    

In [ ]:
print(names_list[783])
print(people_list[4010])

In [ ]:
indx2role

In [ ]:
role_counter = Counter([a for d in indx2role.values() for a in d.values()])

role2color = {n[0]:Category20_20[i] for i,n in enumerate(role_counter.most_common(20))}

In [ ]:
role2color

In [ ]:
for node in G.nodes():
    G.nodes[node]['full_name'] = indx2person[node]

In [ ]:
adams_indxs = {k:v for k,v in person2indx.items() if k.split(" ")[1] == "Adams"}


In [ ]:
adams_indxs

In [ ]:
def merge_nodes(G,nodes, new_node, attr_dict=None):
    """
    Merges the selected `nodes` of the graph G into one `new_node`,
    meaning that all the edges that pointed to or from one of these
    `nodes` will point to or from the `new_node`.
    attr_dict and **attr are defined as in `G.add_node`.
    """
    
    G.add_node(new_node, **attr_dict) # Add the 'merged' node
    edges_to_add = []
    for n1,n2,data in G.edges(data=True):
        # For all edges related to one of the nodes to merge,
        # make an edge going to or coming from the `new gene`.
        if n1 in nodes:
            edges_to_add.append((new_node,n2,data))
        elif n2 in nodes:
            edges_to_add.append((n1,new_node,data))
    
    G.add_edges_from(edges_to_add)
    
    for n in nodes: # remove the merged nodes
        G.remove_node(n)

In [ ]:
def get_subgraph_for_pair(name_1, name_2, G):
    name_1_indx = person2indx[name_1]
    name_2_indx = person2indx[name_2]
    pair = [name_1_indx, name_2_indx]
    nodes = []
    graph_1 = nx.ego_graph(G, name_1_indx)
    graph_2 = nx.ego_graph(G, name_2_indx)
    merge_graph = nx.compose(graph_1, graph_2)
    
    remove_edges = [] 
    for edge in merge_graph.edges():
        if name_1_indx not in edge and name_2_indx not in edge:
            remove_edges.append(edge)
    merge_graph.remove_edges_from(remove_edges)
    
    depositions = set([edge[2]['deposition'] for edge in merge_graph.edges(data=True)])
    
    for node in merge_graph.nodes(data=True):
        print(node)
        print(indx2role[node[0]].keys())
        print(depositions)
        role_key = list(depositions.intersection(set(indx2role[node[0]].keys())))[0]
        node_role = indx2role[node[0]][role_key]
            
        merge_graph.nodes[node[0]]['role'] = node_role
        merge_graph.nodes[node[0]]['role_color'] = role2color[node_role] if node_role in role2color else "black"
        
        if node[0] == name_1_indx or node[0] == name_2_indx:
            merge_graph.nodes[node[0]]['size'] = 0.07
        else:
            merge_graph.nodes[node[0]]['size'] = 0.035

    
    
    print(len(depositions))
    deposition_colors = {}
    scale_i = 0
    for edge in merge_graph.edges(data=True):
        if edge[2]['deposition'] not in deposition_colors:
            deposition_colors[edge[2]['deposition']] = Accent[3][scale_i]
            scale_i += 1
        edge[2]['deposition_color'] = deposition_colors[edge[2]['deposition']]
        edge[2]['line_alpha'] = 1
        
    node_comb = itertools.combinations(merge_graph.nodes, 2)
#     invisible_edges = []
#     for idx_a, idx_b in node_comb:
#         sim = ratio(indx2person[idx_a], indx2person[idx_b])
#         if sim > 0.8:
#             print(indx2person[idx_a], '/', indx2person[idx_b])
# #             sound_a = " ".join(soundex(k) for k in indx2person[idx_a].split(" "))
# #             sound_b = " ".join(soundex(k) for k in indx2person[idx_b].split(" "))
# #             print(sound_a, sound_b)
# #             print('----------')
#             invisible_edges.append((idx_a, idx_b, {'inv' : True, 
#                                                    'similarity' : sim,
#                                                    'line_alpha': 0,
#                                                    'weight': 5 + sim}))
                                   
#     merge_graph.add_edges_from(invisible_edges)
    
    merge_pairs = {}
    for idx_a, idx_b in node_comb:
        sim = ratio(indx2person[idx_a], indx2person[idx_b])
        if sim > 0.8:
            attrs = {
                'full_name' : merge_graph.nodes[idx_a]['full_name'],
                'role' : merge_graph.nodes[idx_a]['role'],
                'role_color' : merge_graph.nodes[idx_a]['role_color'],
                'size' : merge_graph.nodes[idx_a]['size']
                
            }
            merge_pairs[(idx_a, idx_b)] = attrs
    
    for (idx_a, idx_b), attrs in merge_pairs.items():
        merge_nodes(merge_graph, [idx_a, idx_b], idx_a+idx_b, attrs)
    

    return merge_graph

# def get_subgraph_for_pair(name_1, name_2, G):
#     print('Hello')
#     name_1_indx = person2indx[name_1]
#     name_2_indx = person2indx[name_2]
#     ego_g = nx.ego_graph(G, name_1_indx)
#     return ego_g
    
    

In [ ]:
interactions_c_a = Counter()
for adam_name, indx in adams_indxs.items():
    if 'Anselmus' in adam_name:
        for neighbor in G.neighbors(indx):
            interactions_c_a[neighbor] += 1
# interactions_c.most_common()
print(sorted(list(interactions_c_a.keys())))

In [ ]:
# interactions_c_b = Counter()
# for adam_name, indx in adams_indxs.items():
#     if 'Ancelmus' in adam_name:
#         for neighbor in G.neighbors(indx):
#             interactions_c_b[neighbor] += 1
# print(sorted(list(interactions_c_b.keys())))

In [ ]:
# interactions_int = set(list(interactions_c_a.keys())).intersection(set(list(interactions_c_b.keys())))
# for el in interactions_int:
#     print(indx2person[el])

In [ ]:
name_pairs = [('Anselmus Adams', 'Ancelmus Adams'), 
              ('Will: Adams', 'William Adams'), 
              ('Giles Adams', 'Gyles Adams'),
              ('Randle Adams', 'Randall Adams')]
name_pairs = [('Anselmus Adams', 'Ancelmus Adams'), 
              ('Will: Adams', 'William Adams'), 
              ('Giles Adams', 'Gyles Adams')]
subgraphs = [get_subgraph_for_pair(pair[0], pair[1], G) for pair in name_pairs]

In [ ]:
indx2role[3938]

In [ ]:
subgraphs[0].nodes(data=True)

In [ ]:
indx2role[3938]

In [ ]:
pp.pprint(depositions_df.iloc[4010].people_list)
print('------')
pp.pprint(depositions_df.iloc[4186].people_list)
print('------')
pp.pprint(depositions_df.iloc[4932].people_list)

In [ ]:
output_notebook()
plots = []

for i, G in enumerate(subgraphs[:1]):
    
    #Show network
    plot = Plot(plot_width=800, plot_height=800, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
    plot.title.text = "%s vs %s" % name_pairs[i]
    

  # graph_renderer = from_networkx(G, nx.spring_layout, scale=1, k=0.09, center=(0, 0))
  # graph_renderer = from_networkx(G, nx.kamada_kawai_layout, scale=1.0, dim=2)
    graph_renderer = from_networkx(G, nx.spring_layout, k=.19, center=(0, 0))

    graph_renderer.node_renderer.glyph = Circle(radius="size", fill_color="role_color")
    graph_renderer.edge_renderer.glyph = MultiLine(line_color="deposition_color", 
                                                   line_alpha="line_alpha",
                                                   line_width=2)
    
    node_hover_tool = HoverTool(tooltips=[("full_name", "@full_name"), ("role", "@role")])
#     edge_hover_tool = HoverTool(tooltips=[("deposition", "@deposition")])
#     plot.add_tools(node_hover_tool, edge_hover_tool, BoxZoomTool(), ResetTool(), PanTool(), WheelZoomTool())
    plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(), PanTool(), WheelZoomTool())
    
    plot.renderers.append(graph_renderer)


    x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
    node_labels = [node[1]['full_name'] for node in G.nodes(data=True)]
    
    source = ColumnDataSource({'x': x, 'y': y,
                             'full_name': node_labels})


    labels = LabelSet(x='x', 
                    y='y', 
                    text='full_name', 
                    source=source,
                    background_fill_color='white',
                    background_fill_alpha=0.8,
                    text_align='center', 
                    text_baseline='middle',
                    x_offset=5,
                    y_offset=15,
                    text_font_size='0.6em')

    plot.renderers.append(labels)
    plots.append(plot)

grid = gridplot(plots, ncols=3)
show(grid)


# Toponyms

In [ ]:
toponyms_list = depositions_df['creation_place'].tolist()
toponyms_counter = Counter()
for i,d in enumerate(toponyms_list):
    for k,v in d.items():
        toponyms_counter[k+':'+v] += 1
print(toponyms_counter.keys())